In [41]:
import torch 
import numpy as np

In [42]:
w = torch.tensor([2.0], requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)

# Input
x = torch.tensor([3.0])

In [43]:
# Forward pass
y_pred = w * x + b 
print(f"y_pred: {y_pred.item()}")

y_pred: 7.0


In [44]:
# Loss
y_true = torch.tensor([10.0])
loss = (y_pred - y_true) ** 2
print(f"loss: {loss.item()}")

loss: 9.0


In [45]:
print(f"Avant backward: w.grad= {w.grad}")

Avant backward: w.grad= None


In [46]:
loss.backward()

In [47]:
print(f"Après backward: w.grad= {w.grad}")
print(f"Après backward: b.grad= {b.grad}")

Après backward: w.grad= tensor([-18.])
Après backward: b.grad= tensor([-6.])


In [48]:
learning_rate = 0.01
with torch.no_grad(): #Désactiver le tracking des gradients pour l'update w = w + alpha * w
    w -= learning_rate * w.grad
    b -= learning_rate * b.grad

In [49]:
print(f"Nouveaux poids: w= {w}, b= {b}")

Nouveaux poids: w= tensor([2.1800], requires_grad=True), b= tensor([1.0600], requires_grad=True)


In [50]:
# Réinitialiser les gradients (IMPORTANT!)
w.grad.zero_()
b.grad.zero_()
print(f"Après zero_: w.grad= {w.grad}, b.grad= {b.grad}")

Après zero_: w.grad= tensor([0.]), b.grad= tensor([0.])


In [51]:
import torch.nn as nn

perceptron = nn.Linear(1,1)

# Voir les poids initiaux
print(f"Poids: {perceptron.weight}")
print(f"Biais: {perceptron.bias}")

Poids: Parameter containing:
tensor([[-0.2414]], requires_grad=True)
Biais: Parameter containing:
tensor([0.9525], requires_grad=True)


In [52]:
# Forward 
x = torch.tensor([[3.0]]) # Shape (batch_size, features)
print(f"Input: {x}")
y_pred = perceptron(x)
print(y_pred)  

Input: tensor([[3.]])
tensor([[0.2281]], grad_fn=<AddmmBackward0>)


In [53]:
# Loss 
y_true = torch.tensor([[10.0]])
criterion = nn.MSELoss()
loss = criterion(y_pred, y_true)

In [54]:
# Backward
loss.backward()

In [55]:
print(f"Gradient du poids: {perceptron.weight.grad}")
print(f"Gradient du biais: {perceptron.bias.grad}")

Gradient du poids: tensor([[-58.6313]])
Gradient du biais: tensor([-19.5438])


In [56]:
import torch.optim as optim
optimizer = optim.SGD(perceptron.parameters(), lr=0.01)

In [57]:
optimizer.step()

In [58]:
print(f"Nouveaux poids: {perceptron.weight}")
print(f"Nouveaux biais: {perceptron.bias}")

Nouveaux poids: Parameter containing:
tensor([[0.3449]], requires_grad=True)
Nouveaux biais: Parameter containing:
tensor([1.1479], requires_grad=True)


In [59]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4) 
        self.fc2 = nn.Linear(4, 3)
        self.fc3 = nn.Linear(3, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [60]:
model = SimpleNet()

In [61]:
model

SimpleNet(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [62]:
x = torch.tensor([[1.0, 2.0]])
y = torch.tensor([[5.0]])
y_pred = model(x)
loss = nn.MSELoss()(y_pred, y)
loss.backward()

In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize
import matplotlib.pyplot as plt

In [64]:
writer = SummaryWriter()

In [65]:
transform = Compose([
    ToTensor(),
    Normalize((0.1307,), (0.3081,))
])

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

In [66]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [67]:
print(training_data.classes)
print(training_data.data.size())

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([60000, 28, 28])


In [68]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()

        # Convolution 1
        self.conv1 = nn.Conv2d(in_channels, 8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Convolution 2
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)

        # Dropout (désactive 50% des neurones pendant l'entraînement)
        self.dropout = nn.Dropout(p=0.5)

        # Fully-connected
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)

        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = x.reshape(x.size(0), -1)   # Flatten

        x = self.dropout(x)            # <---- Dropout ici !

        x = self.fc1(x)
        return x

In [69]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(device)

model = CNN(in_channels=1, num_classes=10).to(device)
print(model)

mps
CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)


In [70]:
x = torch.rand(1, 1, 28, 28, device=device)
logits = model(x)

In [71]:
logits

tensor([[-0.0478, -0.0280,  0.0608,  0.1290, -0.1025,  0.0620,  0.1067, -0.0313,
         -0.1392, -0.0734]], device='mps:0', grad_fn=<LinearBackward0>)

In [72]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.0956, 0.0975, 0.1065, 0.1141, 0.0905, 0.1067, 0.1116, 0.0972, 0.0872,
         0.0932]], device='mps:0', grad_fn=<SoftmaxBackward0>)

In [73]:
y_pred = pred_probab.argmax(1)
y_pred

tensor([3], device='mps:0')

In [74]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [75]:
def train(dataloader, model, loss_fn, optimizer):
    size= len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss= loss_fn(pred, y)
        writer.add_scalar("Loss/train", loss, batch)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss : {loss:>7f}, [{current:>5d}/{size:>5}]")

In [76]:
def test(dataloader, model, loss_fn):
    size= len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct= 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss+= loss_fn(pred, y).item()
            correct+= (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss/=num_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [77]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
writer.flush()
writer.close()

Epoch 1
---------------------


/Users/laurakn/IIM_mastere/Cours_python/projet/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


loss : 2.347927, [   64/60000]
loss : 0.568152, [ 6464/60000]
loss : 0.454116, [12864/60000]
loss : 0.285067, [19264/60000]
loss : 0.270097, [25664/60000]
loss : 0.112688, [32064/60000]
loss : 0.242265, [38464/60000]
loss : 0.209184, [44864/60000]
loss : 0.259386, [51264/60000]
loss : 0.081331, [57664/60000]
Test Error: 
 Accuracy: 97.3%, Avg loss: 0.088661

Epoch 2
---------------------
loss : 0.111611, [   64/60000]
loss : 0.125626, [ 6464/60000]
loss : 0.395117, [12864/60000]
loss : 0.049341, [19264/60000]
loss : 0.120733, [25664/60000]
loss : 0.088884, [32064/60000]
loss : 0.173945, [38464/60000]
loss : 0.418794, [44864/60000]
loss : 0.072385, [51264/60000]
loss : 0.080817, [57664/60000]
Test Error: 
 Accuracy: 98.1%, Avg loss: 0.062722

Epoch 3
---------------------
loss : 0.094471, [   64/60000]
loss : 0.172807, [ 6464/60000]
loss : 0.134221, [12864/60000]
loss : 0.130909, [19264/60000]
loss : 0.025548, [25664/60000]
loss : 0.062377, [32064/60000]
loss : 0.177363, [38464/60000]
l

In [78]:
example_inputs = (torch.randn(1, 1, 28, 28),)
model.to("cpu")
onnx_program = torch.onnx.export(model, example_inputs,dynamo=True)

[torch.onnx] Obtain model graph for `CNN([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `CNN([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


In [79]:
onnx_program.save("model.onnx")